# API

In [1]:
from sympy import *
from sympy.physics.quantum import *

In [ ]:
n1,n2,n3=symbols('n_1,n_2,n_3')

In [ ]:
g={i:Operator('\gamma_{}'.format(i)) for i in range(1,5)}

In [ ]:
gb={i:Operator(r'\bar{{\gamma}}_{}'.format(i)) for i in range(1,5)}

In [ ]:
c={i:Operator(r'c_{}'.format(i)) for i in range(1,5)}

In [ ]:
cd={i:Dagger(Operator(r'c_{}'.format(i))) for i in range(1,5)}

In [ ]:
def ea(n1,n2,n3):
    return ((1+n1)/(1-n1))**(1/4)* ((n2-I*n3)/sqrt(n2**2+n3**2))**(1/2)
    

In [ ]:
def K(n1,n2,n3,op1,op2):
    expa=N(ea(n1,n2,n3))
    expma=1/expa
    return (1-n1**2)**(1/4)*( (expa+ expma)/2)- ((1-n1**2)**(1/4)*(expa-expma)/2  *I* op1 * op2 )

In [ ]:
selfjoint=[(Dagger(gi),gi) for gi in g.values()]+[(Dagger(gi),gi) for gi in gb.values()]

In [ ]:
squareone=[(gi**2,1) for gi in g.values()]+[(gi**2,1) for gi in gb.values()]

In [ ]:
normalorder=[gi[i] for i in range(1,len(g)+1) for gi in [g, gb]]

In [ ]:
normalorder2=[(normalorder[j]*normalorder[i],-normalorder[i]*normalorder[j]) for i in range(len(normalorder)) for j in range(i+1,len(normalorder))]

In [ ]:
g2c=[(g[i],(c[i]+Dagger(c[i]))) for i in range(1,len(g)+1)]+[(gb[i],(Dagger(c[i])-c[i])/(I)) for i in range(1,len(gb)+1)]

In [ ]:
squarezero=[(ci**2,0) for ci in c.values()] +[(Dagger(ci)**2,0) for ci in c.values()]

In [ ]:
normalorder_C=[Dagger(ci) for ci in c.values()]+[ci for ci in c.values()]

In [ ]:
def delta(i,j):
    return 1 if i%(len(normalorder_C)//2)==j%(len(normalorder_C)//2) else 0

In [ ]:
normalorder2_C=[(normalorder_C[j]*normalorder_C[i],0 if i==j else delta(i,j)-normalorder_C[i]*normalorder_C[j]) for i in range(len(normalorder_C)) for j in range(i,len(normalorder_C))]

In [1]:
def recursiveapply(expr,subs):
    expr1=expr.subs(subs)
    while not expr1== expr:
        expr=expr1
        expr1=expr.subs(subs).expand()
    return expr1

# Op

In [ ]:
Kn=(K(0,0,1,g[1],gb[1]))
# Kn=(K(0,1/sqrt(2),1/sqrt(2),gb[1],g[2]))
# Kn=K(0,.1,sqrt(1-0.1**2),gb[1],g[2])
# Kn=K(.9,0,sqrt(1-0.9**2),gb[1],g[2])

In [ ]:
Kn

In [ ]:
expval=[(Dagger(ci)*ci,1) for ci in c.values()]

In [ ]:
(Dagger(Kn)*Kn).expand().subs(selfjoint).subs(squareone).subs(normalorder2)

In [ ]:
(Kn).expand()

In [ ]:
(gb[1]).subs(g2c).expand().subs(normalorder2_C).expand()

In [ ]:
(I* g[1]* gb[1]).subs(g2c).expand().subs(normalorder2_C).expand()

In [ ]:
(Kn).expand().subs(g2c).expand().subs(normalorder2_C).expand()

## Op

In [ ]:
# Kn=(K(0.9,sqrt(1-0.9**2),0,gb[1],g[2])) *(K(0.9,sqrt(1-0.9**2),0,gb[2],g[3]))
# Kn=(K(0,1/sqrt(2),1/sqrt(2),gb[1],g[2]))
Kn=(K(0.1,sqrt(1-0.1**2)*sin(pi/4),sqrt(1-0.1**2)*cos(pi/4),gb[1],g[2]))


In [ ]:
Kn

In [ ]:
recursiveapply((Dagger(Kn)*Kn).subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*g[1]*gb[1])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*g[1]*g[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*g[1]*gb[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*gb[1]*g[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*gb[1]*gb[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*g[2]*gb[2])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*g[1]*g[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*g[1]*gb[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*g[2]*gb[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*gb[2]*g[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(Kn)*(I*g[3]*gb[3])*Kn).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
Kn.subs(g2c).expand().subs(squarezero).subs(normalorder2_C)

In [ ]:
(Dagger(Kn)*(I*g[1]*gb[1])*Kn).expand().subs(g2c).expand().subs(normalorder2_C).subs(normalorder2_C)

In [ ]:
(Dagger(Kn)*(I*g[1]*g[2])*Kn).expand().subs(g2c).expand().subs(normalorder2_C).subs(normalorder2_C).subs(normalorder2_C)

In [ ]:
(Dagger(Kn)*(I*g[1]*gb[2])*Kn).expand().subs(g2c).expand().subs(normalorder2_C).subs(normalorder2_C)

In [ ]:
N((1+I)**(0.5))

In [ ]:
N(ea(0,1/sqrt(2),1/sqrt(2)))

In [ ]:
N(exp(I*pi/8))

# Initial

In [ ]:
# s=Dagger(c[1]-c[2])/sqrt(2)
s=(1-Dagger(c[1])*Dagger(c[2]))/sqrt(2)

In [ ]:
recursiveapply((Dagger(s)*s).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(s)*I*g[1]*gb[1]*s).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(s)*I*g[1]*g[2]*s).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(s)*I*g[1]*gb[2]*s).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(s)*I*gb[1]*g[2]*s).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(s)*I*gb[1]*gb[2]*s).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(s)*I*g[2]*gb[2]*s).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
s0=Dagger(c[1]-c[2])/sqrt(2)

In [ ]:
recursiveapply((Dagger(s0)*s0).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(s0)*I*g[1]*gb[1]*s0).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(s0)*I*g[1]*g[2]*s0).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((Dagger(s0)*I*g[1]*gb[2]*s0).expand().subs(g2c).expand(),normalorder2_C)

In [ ]:
recursiveapply((g[1]*g[2]*(I*gb[1]*I*gb[2])).expand().subs(g2c).expand(),normalorder2_C)

# Class A

In [ ]:
f={i:Operator(r'f_{}'.format(i)) for i in range(1,5)}
fd={i:Dagger(Operator(r'f_{}'.format(i))) for i in range(1,5)}
g={i:Operator(r'g_{}'.format(i)) for i in range(1,5)}
gd={i:Dagger(Operator(r'g_{}'.format(i))) for i in range(1,5)}

In [ ]:
normalorder_f=[list(op.values()) for op in [fd,gd,f,g]]

In [ ]:
normalorder_f_linear=[normalorder_f[i][j] for i in range(len(normalorder_f)) for j in range(len(normalorder_f[0]))]

In [ ]:
def delta(i,j):
    return 1 if (i%(len(normalorder_f_linear)//2))==(j%(len(normalorder_f_linear)//2)) else 0

In [ ]:
normalorder2_f=[(normalorder_f_linear[j]*normalorder_f_linear[i],0 if i==j else delta(i,j)-normalorder_f_linear[i]*normalorder_f_linear[j]) for i in range(len(normalorder_f_linear)) for j in range(i,len(normalorder_f_linear))]

In [ ]:
def recursiveapply(expr,subs):
    expr1=expr.subs(subs)
    while not expr1== expr:
        expr=expr1
        expr1=expr.subs(subs).expand()
    return expr1

In [ ]:
def commutator(op1,op2,):
    return op1*op2-op2*op1

In [ ]:
recursiveapply(commutator(fd[1]*f[2]+fd[2]*f[1],fd[1]*f[1]+fd[2]*f[2]).expand().subs(normalorder2_f),normalorder2_f)

In [ ]:
recursiveapply(commutator(I*fd[1]*f[2]-I*fd[2]*f[1],fd[1]*f[1]+fd[2]*f[2]).expand().subs(normalorder2_f),normalorder2_f)

In [ ]:
recursiveapply(commutator(fd[1]*f[1]-fd[2]*f[2],fd[1]*f[1]+fd[2]*f[2]).expand().subs(normalorder2_f),normalorder2_f)

In [ ]:
alpha,beta,gamma=symbols('\\alpha,\\beta,\\gamma',real=True)

In [ ]:
Dagger(alpha)

In [ ]:
x=alpha*(fd[1]*f[2]+fd[2]*f[1])
# x=(fd[1]*f[2]+fd[2]*f[1])

In [ ]:
y=beta*(-I*fd[1]*f[2]+I*fd[2]*f[1])
# y=(-I*fd[1]*f[2]+I*fd[2]*f[1])

In [ ]:
z=I*gamma*(fd[1]*f[1]-fd[2]*f[2])
# z=(fd[1]*f[1]-fd[2]*f[2])

In [ ]:
xyz=(alpha-I*beta)*fd[1]*f[2]+(alpha+I*beta)*fd[2]*f[1]+I*gamma*(fd[1]*f[1]-fd[2]*f[2])

In [ ]:
recursiveapply((xyz**1).expand(),normalorder2_f)

In [ ]:
recursiveapply((xyz**2).expand(),normalorder2_f)

In [ ]:
recursiveapply((xyz**3).expand(),normalorder2_f)

In [ ]:
((alpha-I*beta)**3).expand()

In [ ]:
((alpha-1)*(alpha**2+alpha+1)).expand()

In [ ]:
a11,a22,a12,a21,b=symbols('a_11,a_22,a_12,a_21,b',real=True)
A11,A22,A12,A21,B=symbols('A_11,A_22,A_12,A_21,B',real=True)

In [ ]:
rc=recursiveapply(((a11*fd[1]*f[1]+a22*fd[2]*f[2]+a12*fd[1]*f[2]+a21*fd[2]*f[1]+b*(fd[1]*fd[2]*f[1]*f[2]))*(A11*fd[1]*f[1]+A22*fd[2]*f[2]+A12*fd[1]*f[2]+A21*fd[2]*f[1])).expand(),normalorder2_f)

In [ ]:
rc

In [ ]:
recursiveapply((xyz**4).expand(),normalorder2_f)

In [ ]:
tm=Matrix([[A11,A21,0,0,0],[A12,A22,0,0,0],[0,0,A11,A21,0],[0,0,A12,A22,0],[-A22,A21,A12,-A11,A11+A22]])

In [ ]:
init=Matrix([[a11,a12,a21,a22,b]]).T

In [ ]:
((tm@init).dot(basis)-rc).expand()

In [ ]:
rho,phi=symbols('\\rho,\\phi',real=True)

In [ ]:
woU_init=[(a11,0),(a22,0),(a12,rho*exp(-I*phi)),(a21,rho*exp(I*phi)),(b,0)]
woU_M=[(A11,0),(A22,0),(A12,rho*exp(-I*phi)),(A21,rho*exp(I*phi))]
wU_init=[(a11,I*gamma),(a22,-I*gamma),(a12,rho*exp(-I*phi)),(a21,rho*exp(I*phi)),(b,0)]
wU_M=[(A11,I*gamma),(A22,-I*gamma),(A12,rho*exp(-I*phi)),(A21,rho*exp(I*phi))]

In [ ]:
(init).subs(woU_init+woU_M)

In [ ]:
(tm@init).subs(woU_init+woU_M)

In [ ]:
(tm@tm@init).subs(woU_init+woU_M)

In [ ]:
(tm**3@init).subs(woU_init+woU_M)

In [ ]:
(tm**4@init).subs(woU_init+woU_M)

In [ ]:
s1=init.subs(wU_init+wU_M)

In [ ]:
s1

In [ ]:
s2=(tm@s1).subs(wU_init+wU_M)

In [ ]:
s2

In [ ]:
s3=(tm@s2).subs(wU_init+wU_M)

In [ ]:
s3

In [ ]:
s4=((tm@s3).subs(wU_init+wU_M))

In [ ]:
factor(s4)

In [ ]:
s5=((tm@s4).subs(wU_init+wU_M))

In [ ]:
s5

In [ ]:
abs(1+I)

In [ ]:
Ap=sqrt(gamma**2+2*gamma*im(sqrt(rho**2-gamma**2))+abs(rho**2-gamma**2)+rho**2)
Am=sqrt(gamma**2-2*gamma*im(sqrt(rho**2-gamma**2))+abs(rho**2-gamma**2)+rho**2)

In [ ]:
Ap

In [ ]:
Am

In [ ]:
theta=symbols('\\theta',real=True)

In [ ]:
cp=1/sqrt(2)*(exp(-I*phi)*(sech(theta)+I*tanh(theta))*f[1]+f[2])
cp_d=1/sqrt(2)*(exp(I*phi)*(sech(theta)-I*tanh(theta))*fd[1]+fd[2])
cm=1/sqrt(2)*(exp(-I*phi)*(sech(theta)-I*tanh(theta))*f[1]+f[2])
cm_d=1/sqrt(2)*(exp(I*phi)*(sech(theta)+I*tanh(theta))*fd[1]+fd[2])

In [ ]:
recursiveapply((cp_d*cp-cm_d*cm).expand(),normalorder2_f)

In [ ]:
(cp_d*cp-cm_d*cm).expand()

In [ ]:
recursiveapply((cp_d*cp-cm_d*cm).expand(),normalorder2_f)

In [ ]:
eye(2)

In [ ]:
pm={0:eye(2),1:Matrix([[0,1],[1,0]]),2:Matrix([[0,-I],[I,0]]),3:Matrix([[1,0],[0,-1]])}

In [ ]:
TensorProduct(pm[1],pm[1])

In [ ]:
Matrix([fd[1],fd[2],gd[1],gd[2]]).T@TensorProduct(pm[1],pm[0])@Matrix([f[1],f[2],g[1],g[2]])

In [ ]:
Matrix([fd[1],fd[2],gd[1],gd[2]]).T@TensorProduct(pm[3],pm[0])@Matrix([f[1],f[2],g[1],g[2]])

In [ ]:
a = {(i, j): symbols(f'a_{{{i}{j}}}',real=True) for i in range(4) for j in range(4)}


In [393]:
a_subs=[(val,np.random.rand()) for key,val in a.items()]

In [384]:
coef={(i,j):a[i,j] if i in [1,2] else I*a[i,j] for i in range(4) for j in range(4)}

In [ ]:
exp_sigma=[coef[i,j]*TensorProduct(pm[i],pm[j]) for i in range(4) for j in range(4)]

In [ ]:
import numpy as np
import scipy

In [391]:
exp_sigma_nU=[coef[i,j]*TensorProduct(pm[i],pm[j]) for i in range(4) for j in range(4) if i in [1,2]]
exp_sigma_U=[coef[i,j]*TensorProduct(pm[i],pm[j]) for i in range(4) for j in range(4) if i in [0,3]]


In [395]:
sigmaU=scipy.linalg.expm(np.array((Add(*exp_sigma_U).subs(a_subs)).tolist()).astype(complex))@scipy.linalg.expm(np.array((Add(*exp_sigma_nU).subs(a_subs)).tolist()).astype(complex))

In [366]:
# sigmaU=scipy.linalg.expm(np.array((Add(*exp_sigma).subs(a_subs)).tolist()).astype(complex))

In [396]:
np.round(sigmaU.T.conj()@np.kron(np.diag([1,-1]),np.eye(2))@sigmaU,3)

array([[ 1.+0.j, -0.+0.j, -0.+0.j,  0.-0.j],
       [-0.-0.j,  1.+0.j, -0.-0.j,  0.-0.j],
       [-0.-0.j, -0.+0.j, -1.+0.j,  0.+0.j],
       [ 0.+0.j,  0.+0.j,  0.+0.j, -1.+0.j]])

In [ ]:
sigmaU

In [ ]:
exp(sigmaU).H

In [ ]:
(exp(sigmaU).H@TensorProduct(pm[0],pm[3])@exp(sigmaU))

In [397]:
# exp_herm=[coef[i,j]*(Matrix([fd[1],fd[2],gd[1],gd[2]]).T@TensorProduct(pm[i],pm[j])@Matrix([f[1],f[2],g[1],g[2]]))[0,0] for i in range(4) for j in range(4)]
exp_herm=[coef[i,j]*(Matrix([fd[1],fd[2],gd[1],gd[2]]).T@TensorProduct(pm[i],pm[j])@Matrix([f[1],f[2],g[1],g[2]]))[0,0] for i in range(4) for j in range(4) if i in [1,2]]

In [399]:
exp_herm_simp=simplify(sum(exp_herm).expand())

In [400]:
exp_herm_simp

a_{10}*Dagger(f_1)*g_1 + a_{10}*Dagger(f_2)*g_2 + a_{10}*Dagger(g_1)*f_1 + a_{10}*Dagger(g_2)*f_2 + a_{11}*Dagger(f_1)*g_2 + a_{11}*Dagger(f_2)*g_1 + a_{11}*Dagger(g_1)*f_2 + a_{11}*Dagger(g_2)*f_1 - I*a_{12}*Dagger(f_1)*g_2 + I*a_{12}*Dagger(f_2)*g_1 - I*a_{12}*Dagger(g_1)*f_2 + I*a_{12}*Dagger(g_2)*f_1 + a_{13}*Dagger(f_1)*g_1 - a_{13}*Dagger(f_2)*g_2 + a_{13}*Dagger(g_1)*f_1 - a_{13}*Dagger(g_2)*f_2 - I*a_{20}*Dagger(f_1)*g_1 - I*a_{20}*Dagger(f_2)*g_2 + I*a_{20}*Dagger(g_1)*f_1 + I*a_{20}*Dagger(g_2)*f_2 - I*a_{21}*Dagger(f_1)*g_2 - I*a_{21}*Dagger(f_2)*g_1 + I*a_{21}*Dagger(g_1)*f_2 + I*a_{21}*Dagger(g_2)*f_1 - a_{22}*Dagger(f_1)*g_2 + a_{22}*Dagger(f_2)*g_1 + a_{22}*Dagger(g_1)*f_2 - a_{22}*Dagger(g_2)*f_1 - I*a_{23}*Dagger(f_1)*g_1 + I*a_{23}*Dagger(f_2)*g_2 + I*a_{23}*Dagger(g_1)*f_1 - I*a_{23}*Dagger(g_2)*f_2

In [401]:
collect_coeff(exp_herm_simp)

(a_{10} - a_{13} - I*a_{20} + I*a_{23})*Dagger(f_2)*g_2 + (a_{10} - a_{13} + I*a_{20} - I*a_{23})*Dagger(g_2)*f_2 + (a_{10} + a_{13} - I*a_{20} - I*a_{23})*Dagger(f_1)*g_1 + (a_{10} + a_{13} + I*a_{20} + I*a_{23})*Dagger(g_1)*f_1 + (a_{11} - I*a_{12} - I*a_{21} - a_{22})*Dagger(f_1)*g_2 + (a_{11} - I*a_{12} + I*a_{21} + a_{22})*Dagger(g_1)*f_2 + (a_{11} + I*a_{12} - I*a_{21} + a_{22})*Dagger(f_2)*g_1 + (a_{11} + I*a_{12} + I*a_{21} - a_{22})*Dagger(g_2)*f_1

In [403]:
Add(*exp_sigma_nU)

Matrix([
[                                    0,                                     0, a_{10} + a_{13} - I*a_{20} - I*a_{23}, a_{11} - I*a_{12} - I*a_{21} - a_{22}],
[                                    0,                                     0, a_{11} + I*a_{12} - I*a_{21} + a_{22}, a_{10} - a_{13} - I*a_{20} + I*a_{23}],
[a_{10} + a_{13} + I*a_{20} + I*a_{23}, a_{11} - I*a_{12} + I*a_{21} + a_{22},                                     0,                                     0],
[a_{11} + I*a_{12} + I*a_{21} - a_{22}, a_{10} - a_{13} + I*a_{20} - I*a_{23},                                     0,                                     0]])

In [ ]:
def collect_coeff(expr):
    assert expr.func==Add, 'Only support addition'
    # Assume the order is always (commutative, non-commutative)
    terms={}

    for arg in expr.args:
        for i in range(0,len(arg.args)):
            if not arg.args[i].is_commutative:
                break
        commutative = Mul(*arg.args[:i]) if i >0 else 1
        noncommutative = Mul(*arg.args[i:])
        if noncommutative in terms:
            terms[noncommutative]+=commutative
        else:
            terms[noncommutative]=commutative
    rs=0
    for key,val in terms.items():
        rs+=val*key
    return rs




In [ ]:
exp_herm[7]

In [ ]:
cp=(exp(-I*phi)*(I*gamma+sqrt(rho**2-gamma**2))*f[1]+rho*f[2])
cp_d=(exp(I*phi)*(-I*gamma+conjugate(sqrt(rho**2-gamma**2)))*fd[1]+rho*fd[2])
cm=(exp(-I*phi)*(I*gamma-sqrt(rho**2-gamma**2))*f[1]+rho*f[2])
cm_d=(exp(I*phi)*(-I*gamma-conjugate(sqrt(rho**2-gamma**2)))*fd[1]+rho*fd[2])

In [ ]:
cp=1/Ap*(exp(-I*phi)*(I*gamma+sqrt(rho**2-gamma**2))*f[1]+rho*f[2])
cp_d=1/Ap*(exp(I*phi)*(-I*gamma+conjugate(sqrt(rho**2-gamma**2)))*fd[1]+rho*fd[2])
cm=1/Am*(exp(-I*phi)*(I*gamma-sqrt(rho**2-gamma**2))*f[1]+rho*f[2])
cm_d=1/Am*(exp(I*phi)*(-I*gamma-conjugate(sqrt(rho**2-gamma**2)))*fd[1]+rho*fd[2])

In [ ]:
cp

In [ ]:
cp_d

In [ ]:
cm

In [ ]:
cm_d

In [ ]:
lp=sqrt(rho**2-gamma**2)
lm=-sqrt(rho**2-gamma**2)

In [ ]:
recursiveapply((cp_d*cp).expand(),normalorder2_f)

In [ ]:
recursiveapply(commutator(cp_d*cp,cm_d*cm).expand(),normalorder2_f)

In [ ]:
lp*cp_d*cp+lm*cm_d*cm

In [ ]:
recursiveapply((lp*cp_d*cp+lm*cm_d*cm).expand(),normalorder2_f)

In [ ]:
basis=Matrix([(fd[1]*f[1]),(fd[1]*f[2]),(fd[2]*f[1]),(fd[2]*f[2]),(fd[1]*fd[2]*f[1]*f[2])])

In [ ]:
basis

In [ ]:
phi=Symbol(r'\phi',real=True)

In [ ]:
fp_d=1/sqrt(2)*(fd[1]+exp(-I*phi)*fd[2])
fp=1/sqrt(2)*(f[1]+exp(I*phi)*f[2])
fm_d=1/sqrt(2)*(fd[1]-exp(-I*phi)*fd[2])
fm=1/sqrt(2)*(f[1]-exp(I*phi)*f[2])

In [ ]:
(fp_d*fp+fm_d*fm).expand()

In [ ]:
recursiveapply(commutator(fp_d*fp,fm_d*fm).expand(),normalorder2_f)

In [ ]:
recursiveapply(((fp_d*fp)**2-(fp_d*fp)).expand(),normalorder2_f)

In [ ]:
def group_noncommutative(expr, op):
    # Ensure the expression is expanded so we can iterate through terms
    expr = expr.expand()
    
    # Separate the terms involving the operator `op`
    op_terms = [term for term in expr.args if op in term.free_symbols]
    
    # Separate the terms not involving the operator `op`
    other_terms = [term for term in expr.args if op not in term.free_symbols]
    
    # Sum the coefficients of the terms involving `op`
    op_coeff = sum(term.coeff(op) for term in op_terms)
    
    # Reconstruct the grouped expression
    grouped_expr = op_coeff*op + Add(*other_terms)
    
    return grouped_expr

In [ ]:
group_noncommutative(alpha*f[1]+beta*f[1]+ gamma*Dagger(f[1]), f[1])

In [ ]:
recursiveapply(commutator(x,y).expand()-2*I*z,normalorder2_f)

In [ ]:
recursiveapply(commutator(y,z).expand()-2*I*x,normalorder2_f)

In [ ]:
recursiveapply(commutator(z,x).expand()-2*I*y,normalorder2_f)

In [ ]:
z

In [ ]:
xy=recursiveapply(commutator(x+y+z,x+y-z).expand(),normalorder2_f)

In [ ]:
xy

In [ ]:
(x+y + (xy/2).simplify())

In [ ]:
xy3=recursiveapply(commutator(x-y,xy).expand(),normalorder2_f)

In [ ]:
xy3

In [ ]:
x+y

In [ ]:
xy

In [ ]:
x.subs()

In [ ]:
recursiveapply(commutator(x,y).expand(),normalorder2_f)

In [ ]:
z

In [ ]:
recursiveapply(commutator(fd[1]*f[1]-fd[2]*f[2],fd[1]*f[1]+fd[2]*f[2]).expand().subs(normalorder2_f),normalorder2_f)

In [ ]:
recursiveapply(((fd[1]+gd[1])*f[1]*fd[1]*g[1]*gd[1]*(f[1]+g[1])).expand(),normalorder2_f)

In [ ]:
recursiveapply(((fd[1]-gd[1])*f[1]*fd[1]*g[1]*gd[1]*(f[1]-g[1])).expand(),normalorder2_f)

In [ ]:
recursiveapply(((1+fd[1]*gd[1])*f[1]*fd[1]*g[1]*gd[1]*(1+g[1]*f[1])).expand(),normalorder2_f)

In [ ]:
recursiveapply(((1-fd[1]*gd[1])*f[1]*fd[1]*g[1]*gd[1]*(1-g[1]*f[1])).expand(),normalorder2_f)

In [ ]:
recursiveapply((gd[1]*f[1]+fd[1]*g[1]).expand(),normalorder2_f)

In [ ]:
recursiveapply(((gd[1]*f[1]+fd[1]*g[1])**2).expand(),normalorder2_f)

In [ ]:
recursiveapply(((gd[1]*f[1]+fd[1]*g[1])**3).expand(),normalorder2_f)

In [ ]:
recursiveapply(((gd[1]*f[1]+fd[1]*g[1])**4).expand(),normalorder2_f)

In [ ]:
((gd[1]*f[1]+fd[1]*g[1])**2).expand()

In [ ]:
((gd[1]*f[1]+fd[1]*g[1])**3).expand()

In [ ]:
recursiveapply(((-I*gd[1]*f[1]+I*fd[1]*g[1])*(gd[1]*f[1]+fd[1]*g[1])- (gd[1]*f[1]+fd[1]*g[1])*(-I*gd[1]*f[1]+I*fd[1]*g[1])).expand(),normalorder2_f)

In [ ]:
recursiveapply((-I*gd[1]*f[1]+I*fd[1]*g[1]).expand(),normalorder2_f)

In [ ]:
recursiveapply(((-I*gd[1]*f[1]+I*fd[1]*g[1])**2).expand(),normalorder2_f)

In [ ]:
recursiveapply(((-I*gd[1]*f[1]+I*fd[1]*g[1])**3).expand(),normalorder2_f)

In [ ]:
recursiveapply(((-I*gd[1]*f[1]+I*fd[1]*g[1])**4).expand(),normalorder2_f)

In [ ]:
alpha,beta,=symbols(r'\alpha,\beta')

In [ ]:
alpha

In [ ]:
exp1=(1+(cosh(alpha)-1)*(2*fd[1]*gd[1]*f[1]*g[1]+fd[1]*f[1]+gd[1]*g[1])+sinh(alpha)*(fd[1]*g[1]+gd[1]*f[1]))

In [ ]:
exp2=(1+(cosh(beta)-1)*(2*fd[1]*gd[1]*f[1]*g[1]+fd[1]*f[1]+gd[1]*g[1])+sinh(beta)*(I*fd[1]*g[1]-I*gd[1]*f[1]))

In [ ]:
recursiveapply((exp1*exp2).expand(),normalorder2_f)

In [ ]:
recursiveapply((exp1*exp2).expand(),normalorder2_f).simplify()

# Gamma

In [2]:
from sympy import *
from sympy.physics.quantum import *

In [3]:
g={i:Operator('\gamma_{}'.format(i)) for i in range(2)}
eta={i:Operator('\eta_{}'.format(i)) for i in range(2)}
xi={i:Operator(r'\xi_{}'.format(i)) for i in range(2)}

In [4]:
n=Symbol('n')

In [5]:
selfjoint=[(Dagger(gi),gi) for gi in list(g.values())+list(eta.values())+list(xi.values())]

In [6]:
squareone=[(gi**2,1) for gi in list(g.values())+list(eta.values())+list(xi.values())]

In [7]:
normalorder=[gi[i] for gi in [xi,eta,g] for i in range(len(g))]

In [8]:
normalorder2=[(normalorder[j]*normalorder[i],-normalorder[i]*normalorder[j])  if j>i else (normalorder[j]*normalorder[i],1) for i in range(len(normalorder)) for j in range(i,len(normalorder))]

In [37]:
ket0=(g[0]-I*g[1])/2
bra0=(eta[0]+I*eta[1])/2
ket1=(g[0]+I*g[1])/2
bra1=(eta[0]-I*eta[1])/2

In [9]:
tensor=(1+I*xi[0]*xi[1])/2*(1-I*eta[0]*eta[1])/2*(1+I*g[0]*g[1])/2

In [10]:
tensor0=(1-I*eta[0]*eta[1])/2*(1+I*g[0]*g[1])/2

In [11]:
tensor0_p=(1+I*eta[0]*eta[1])/2*(1+I*g[0]*g[1])/2

In [145]:
p=(1+I*g[0]*eta[0])/2 * (1+I*g[1]*eta[1])/2

In [43]:
recursiveapply((p*(ket0+ket1)*(bra0+bra1)).expand(),normalorder2)

-I/4 - I*\eta_0*\eta_1*\gamma_0*\gamma_1/4 - \eta_0*\gamma_0/4 - \eta_1*\gamma_1/4

In [210]:
identity=(((xi[0]-I*xi[1]))*((eta[0]+I*eta[1])) *1/2+((xi[0]+I*xi[1]))*((eta[0]-I*eta[1]))*1/2)/sqrt(2)

In [211]:
identity.expand()

sqrt(2)*\xi_0*\eta_0/2 + sqrt(2)*\xi_1*\eta_1/2

In [212]:
rho=recursiveapply((identity*Dagger(identity)).expand().subs(selfjoint),normalorder2)

In [237]:
c_xi=Symbol('c_\xi')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \xXX escape (1627185929.py, line 1)

In [ ]:
(1+c_xi)

In [213]:
rho

1 + \xi_0*\xi_1*\eta_0*\eta_1

In [175]:
recursiveapply((p*identity*ket0).expand(),normalorder2)

I*\xi_0/16 - \xi_0*\eta_0*\eta_1/16 + I*\xi_0*\eta_0*\eta_1*\gamma_0*\gamma_1/16 + \xi_0*\eta_0*\gamma_0/16 - I*\xi_0*\eta_0*\gamma_1/16 + I*\xi_0*\eta_1*\gamma_0/16 + \xi_0*\eta_1*\gamma_1/16 + \xi_0*\gamma_0*\gamma_1/16 + \xi_1/16 + I*\xi_1*\eta_0*\eta_1/16 + \xi_1*\eta_0*\eta_1*\gamma_0*\gamma_1/16 - I*\xi_1*\eta_0*\gamma_0/16 - \xi_1*\eta_0*\gamma_1/16 + \xi_1*\eta_1*\gamma_0/16 - I*\xi_1*\eta_1*\gamma_1/16 - I*\xi_1*\gamma_0*\gamma_1/16

In [176]:
recursiveapply((p*identity*ket1).expand(),normalorder2)

I*\xi_0/16 + \xi_0*\eta_0*\eta_1/16 + I*\xi_0*\eta_0*\eta_1*\gamma_0*\gamma_1/16 + \xi_0*\eta_0*\gamma_0/16 + I*\xi_0*\eta_0*\gamma_1/16 - I*\xi_0*\eta_1*\gamma_0/16 + \xi_0*\eta_1*\gamma_1/16 - \xi_0*\gamma_0*\gamma_1/16 - \xi_1/16 + I*\xi_1*\eta_0*\eta_1/16 - \xi_1*\eta_0*\eta_1*\gamma_0*\gamma_1/16 + I*\xi_1*\eta_0*\gamma_0/16 - \xi_1*\eta_0*\gamma_1/16 + \xi_1*\eta_1*\gamma_0/16 + I*\xi_1*\eta_1*\gamma_1/16 - I*\xi_1*\gamma_0*\gamma_1/16

In [41]:
recursiveapply((p*ket1*bra0).expand(),normalorder2)

0

In [42]:
recursiveapply((p*ket1*bra1).expand(),normalorder2)

-I/8 - \eta_0*\eta_1/8 - I*\eta_0*\eta_1*\gamma_0*\gamma_1/8 - \eta_0*\gamma_0/8 - I*\eta_0*\gamma_1/8 + I*\eta_1*\gamma_0/8 - \eta_1*\gamma_1/8 + \gamma_0*\gamma_1/8

In [36]:
recursiveapply(((xi[0]-I*xi[1])/2 * (xi[0]+I*xi[1])/2).expand(),normalorder2)

1/2 + I*\xi_0*\xi_1/2

In [54]:
Gamma

(\xi_0 - I*\xi_1)*(\eta_0 + I*\eta_1)/4

In [57]:
bra0

(\eta_0 + I*\eta_1)/2

In [56]:
ket0

(\gamma_0 - I*\gamma_1)/2

In [58]:
recursiveapply((p*Gamma*ket0).expand(),normalorder2)

I*\xi_0/16 - \xi_0*\eta_0*\eta_1/16 + I*\xi_0*\eta_0*\eta_1*\gamma_0*\gamma_1/16 + \xi_0*\eta_0*\gamma_0/16 - I*\xi_0*\eta_0*\gamma_1/16 + I*\xi_0*\eta_1*\gamma_0/16 + \xi_0*\eta_1*\gamma_1/16 + \xi_0*\gamma_0*\gamma_1/16 + \xi_1/16 + I*\xi_1*\eta_0*\eta_1/16 + \xi_1*\eta_0*\eta_1*\gamma_0*\gamma_1/16 - I*\xi_1*\eta_0*\gamma_0/16 - \xi_1*\eta_0*\gamma_1/16 + \xi_1*\eta_1*\gamma_0/16 - I*\xi_1*\eta_1*\gamma_1/16 - I*\xi_1*\gamma_0*\gamma_1/16

In [ ]:
recursiveapply((p*).expand(),normalorder2)

In [15]:
recursiveapply((p*tensor0).expand(),normalorder2)

1/8 - I*\eta_0*\eta_1/8 + \eta_0*\eta_1*\gamma_0*\gamma_1/8 - I*\eta_0*\gamma_0/8 + \eta_0*\gamma_1/8 - \eta_1*\gamma_0/8 - I*\eta_1*\gamma_1/8 + I*\gamma_0*\gamma_1/8

In [56]:
tensor_neg=(1-I*xi[0]*xi[1])/2*(1+I*eta[0]*eta[1])/2*(1-I*g[0]*g[1])/2

In [57]:
recursiveapply((p*tensor0_p+p*tensor_neg).expand(),normalorder2)

1/16 + I*\eta_0*\eta_1/16 + \eta_0*\eta_1*\gamma_0*\gamma_1/16 - I*\eta_0*\gamma_0/16 - \eta_0*\gamma_1/16 + \eta_1*\gamma_0/16 - I*\eta_1*\gamma_1/16 - I*\gamma_0*\gamma_1/16 - I*\xi_0*\xi_1/16 + \xi_0*\xi_1*\eta_0*\eta_1/16 - I*\xi_0*\xi_1*\eta_0*\eta_1*\gamma_0*\gamma_1/16 - \xi_0*\xi_1*\eta_0*\gamma_0/16 + I*\xi_0*\xi_1*\eta_0*\gamma_1/16 - I*\xi_0*\xi_1*\eta_1*\gamma_0/16 - \xi_0*\xi_1*\eta_1*\gamma_1/16 - \xi_0*\xi_1*\gamma_0*\gamma_1/16

In [52]:
recursiveapply((p*tensor).expand(),normalorder2)

1/16 - I*\eta_0*\eta_1/16 + \eta_0*\eta_1*\gamma_0*\gamma_1/16 - I*\eta_0*\gamma_0/16 + \eta_0*\gamma_1/16 - \eta_1*\gamma_0/16 - I*\eta_1*\gamma_1/16 + I*\gamma_0*\gamma_1/16 + I*\xi_0*\xi_1/16 + \xi_0*\xi_1*\eta_0*\eta_1/16 + I*\xi_0*\xi_1*\eta_0*\eta_1*\gamma_0*\gamma_1/16 + \xi_0*\xi_1*\eta_0*\gamma_0/16 + I*\xi_0*\xi_1*\eta_0*\gamma_1/16 - I*\xi_0*\xi_1*\eta_1*\gamma_0/16 + \xi_0*\xi_1*\eta_1*\gamma_1/16 - \xi_0*\xi_1*\gamma_0*\gamma_1/16

In [ ]:
recursiveapply((p*tensor).expand(),normalorder2)

In [49]:
recursiveapply((g[1]*eta[1]*eta[0]*eta[1]*g[0]*g[1]),normalorder2)

-\eta_0*\gamma_0

In [50]:
recursiveapply((g[0]*eta[0]*g[1]*eta[1]*eta[0]*eta[1]),normalorder2)

\gamma_0*\gamma_1

In [42]:
recursiveapply(((1-I*eta[0]*eta[1])*(1+n*I*g[0]*g[1])).expand(),normalorder2)

2 + 2*I*\gamma_0*\gamma_1

In [225]:
Gamma=(1-I*xi[0]*xi[1])/2*(1+I*eta[0]*eta[1])/2
# Gamma=(xi[0]-I*xi[1])/2 * (eta[0]+I*eta[1])/2+ (xi[0]+I*xi[1])/2 * (eta[0]-I*eta[1])/2
# Gamma=(
#     (1+I*xi[0]*xi[1])/2*(1-I*eta[0]*eta[1])/2 
#  +(1-I*xi[0]*xi[1])/2*(1+I*eta[0]*eta[1])/2 
#  +I*(xi[0]-I*xi[1])/2 * (eta[0]+I*eta[1])/2
#  +I*(xi[0]+I*xi[1])/2 * (eta[0]-I*eta[1])/2
   # )

# Gamma=rho

In [226]:
simp=recursiveapply(Gamma.expand(),normalorder2)

In [227]:
simp

1/4 + I*\eta_0*\eta_1/4 - I*\xi_0*\xi_1/4 + \xi_0*\xi_1*\eta_0*\eta_1/4

In [228]:
recursiveapply(Dagger(simp).subs(selfjoint),normalorder2)

1/4 + I*\eta_0*\eta_1/4 - I*\xi_0*\xi_1/4 + \xi_0*\xi_1*\eta_0*\eta_1/4

In [229]:
recursiveapply((Gamma).expand(),normalorder2)

1/4 + I*\eta_0*\eta_1/4 - I*\xi_0*\xi_1/4 + \xi_0*\xi_1*\eta_0*\eta_1/4

In [230]:
recursiveapply((I*eta[0]*eta[0]*Gamma).expand(),normalorder2)

I/4 - \eta_0*\eta_1/4 + \xi_0*\xi_1/4 + I*\xi_0*\xi_1*\eta_0*\eta_1/4

In [231]:
recursiveapply((I*eta[0]*eta[1]*Gamma).expand(),normalorder2) 

1/4 + I*\eta_0*\eta_1/4 - I*\xi_0*\xi_1/4 + \xi_0*\xi_1*\eta_0*\eta_1/4

In [232]:
recursiveapply((I*eta[0]*xi[0]*Gamma).expand(),normalorder2) 

-I*\xi_0*\eta_0/4 + \xi_0*\eta_1/4 - \xi_1*\eta_0/4 - I*\xi_1*\eta_1/4

In [233]:
recursiveapply((I*eta[0]*xi[1]*Gamma).expand(),normalorder2) 

\xi_0*\eta_0/4 + I*\xi_0*\eta_1/4 - I*\xi_1*\eta_0/4 + \xi_1*\eta_1/4

In [234]:
recursiveapply((I*eta[1]*xi[0]*Gamma).expand(),normalorder2) 

-\xi_0*\eta_0/4 - I*\xi_0*\eta_1/4 + I*\xi_1*\eta_0/4 - \xi_1*\eta_1/4

In [235]:
recursiveapply((I*eta[1]*xi[1]*Gamma).expand(),normalorder2) 

-I*\xi_0*\eta_0/4 + \xi_0*\eta_1/4 - \xi_1*\eta_0/4 - I*\xi_1*\eta_1/4

In [236]:
recursiveapply((I*xi[0]*xi[1]*Gamma).expand(),normalorder2) 

-1/4 - I*\eta_0*\eta_1/4 + I*\xi_0*\xi_1/4 - \xi_0*\xi_1*\eta_0*\eta_1/4

In [72]:
(I*eta[0]*eta[1])*(1-eta[0]*eta[1])/2

I*\eta_0*\eta_1*(1 - \eta_0*\eta_1)/2

In [73]:
recursiveapply(((I*eta[0]*eta[1])*(1-I*eta[0]*eta[1])/2).expand(),normalorder2)

-1/2 + I*\eta_0*\eta_1/2

In [74]:
recursiveapply(((I*eta[0]*eta[1])*(1+I*eta[0]*eta[1])/2).expand(),normalorder2)

1/2 + I*\eta_0*\eta_1/2